In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from surprise import Dataset, Reader, accuracy, NormalPredictor, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from surprise.accuracy import rmse
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor
from IPython.core.display import HTML
from surprise.model_selection.split import train_test_split as surprise_train_test_split
from surprise.model_selection import GridSearchCV, cross_validate

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.collocations import *
import re
import nltk

%matplotlib inline

In [12]:
user_ratings_df = pd.read_csv("./Data/user_reviews_no_zero.csv", index_col=[0])

In [10]:
recipes_df = pd.read_csv("./Data/recipes_cleaned.csv", index_col=[0])

In [19]:
recipes_df.head()

,Unnamed: 0,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,...,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv,recipe_type
0,0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,NaN
1,1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,NaN
2,2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,NaN
3,3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,...,"['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,NaN
4,4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,NaN


## Setting up Surprise

In [13]:
user_ratings_df[["user_id", "recipe_id"]] = user_ratings_df[["user_id", "recipe_id"]].astype(str)

In [14]:
rating_surprise_df = user_ratings_df[["user_id", "recipe_id", "rating"]]

In [15]:
reader = Reader(rating_scale=(1,5))
surprise_data = Dataset.load_from_df(rating_surprise_df, reader)

In [16]:
trainset_full = surprise_data.build_full_trainset()

In [17]:
best_model = SVD(n_factors = 10 ,  n_epochs= 30  , lr_all = .0025, random_state=42)
best_model.fit(trainset_full)

In [24]:
## Subset data frame to show reviewers the products they have rated 

prior_ratings_df = pd.DataFrame(user_ratings_df.set_index("user_id"))
prior_ratings_df.drop(columns=["date", "rating", "review"], inplace=True)
prior_ratings_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 537267 entries, 56680 to 1122988
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   recipe_id  537267 non-null  object
dtypes: object(1)
memory usage: 8.2+ MB


In [29]:
prior_ratings_df.head(30)

,recipe_id
user_id,
56680,79222
183565,79222
101823,79222
446143,79222
226989,79222
868654,79222
302867,79222
930021,79222
241697,79222


In [22]:
user_ratings_df.head()

,user_id,recipe_id,date,rating,review
10,56680,79222,2006-11-11,5.0,"Oh, This was wonderful! Had a soup and salad dinner and most people barely ate salad, LOL. This doubled easily and reheats well too!"
11,183565,79222,2006-02-13,5.0,"Wow! My family loves this recipe and it is a great winter meal. I do have to say that I adjusted a little but the base remains the same. Here's what I did different: I peeled my potatoes and diced them very small. I left out the creamed corn, nutmeg, cayenne pepper, and parsley. Instead I used approx. 1 tsp of Old Bay seasoning. Then right before I stirred in the crab meat, I used my potato masher to mash up some of the potatoes. This makes the chowder a little thicker. These are all just my personal preferences. I have made this several times and have forgotten to review it. It's awesome! Thanks Pantry Chef Starkey."
12,101823,79222,2006-03-21,5.0,"Excellent chowder. This was the perfect warm-up at the end of a cold, snowy day. I used more crab meat than the recipe called for, but otherwise followed the directions. This was so simple, and worked well to put in the crockpot to keep warm while I waited for Hubby to come home for the evening. We will make this again."
13,446143,79222,2008-02-01,4.0,"Oh, how wonderful! I doubled the crab, and added some carrots (and some extra broth to compensate.) We still thought there should be more crab though. The nutmeg flavor was wonderful and the cayenne added a tiny kick. Lovely recipe PCS!"
14,226989,79222,2008-03-07,4.0,"DH and I enjoyed this. However I used it only as a base for a potato/seafood chowder. I made as is, except that I added about 1/4lb bacon, which I diced and fryed, tossed with 8oz crab the recipe called for and added about 1 cup pre cooked shirmp. Fryed them all together for a moment and put that all into the chowder. I also used a small can of creamed corn, as I'm not a huge fan of it. I think it needs salt for sure, I also increase the nutmeg and cayenne and added a bit of a chilli/garlic sauce. Good soup, I'll use it again."


In [52]:
def recommended_recipes():
    pd.set_option("display.max_colwidth", None)
    user = input("user_id: ")
    #ingredient_request = input("What ingredients do you need to use?  ")
    #n_ingredients = int(input("How many ingredients do you want to match? "))
    
    reviewed = list(prior_ratings_df.loc[user,"recipe_id"])
    not_reviewed = recipes_df.copy()
    not_reviewed = not_reviewed[not_reviewed.recipe_id.isin(reviewed) == False]
    not_reviewed.reset_index(inplace=True)
    
    #not_reviewed = not_reviewed[not_reviewed["ingredients"].isin(ingredient_request) == True]
    
    not_reviewed["predicted_rating"] = not_reviewed["recipe_id"].apply(lambda x: best_model.predict(user, x).est)
    not_reviewed.sort_values(by="predicted_rating", ascending=False, inplace=True)
    not_reviewed = not_reviewed[["name", "minutes", "steps", "ingredients", "recipe_type", "predicted_rating"]] 
    
    return not_reviewed.head(25)
    
    

user_id: 446143


,name,minutes,steps,ingredients,recipe_type,predicted_rating
0,arriba baked winter squash mexican style,55,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",NaN,4.855004
154183,pat s chiles rellenos squares,60,"['combine all ingredients , mixing well', 'pour into a 9 x 13-inch casserole dish', 'bake 50 to 60 minutes at 325f', 'optional: add small can sliced black olives and / or cover top with thinly sliced tomatoes as soon as it is baked']","['diced green chilies', 'longhorn cheese', 'monterey jack cheese', 'low-fat evaporated milk', 'flour', 'eggs', 'black olives', 'fresh tomato', 'picante sauce']",NaN,4.855004
154213,pat s spicy fried wings,100,"['combine ingredients for dipping sauce and chill for 30 minutes or more to incorporate flavors', 'whisk together the seasoning salt , red pepper flakes , black pepper , cayenne pepper , poultry seasoning and lemon pepper , reserving 2 tablespoons separately', 'rinse chicken wings in cold water , pat dry , and place in a shallow baking sheet', 'sprinkle with the seasoning and let them marinate , refrigerated and covered with plastic wrap , for 1 hour', 'whisk eggs , hot sauce , red pepper flakes , black pepper and cayenne pepper into a mixing bowl and set aside', 'place flour in a shallow plate', 'heat cooking oil for deep frying to 350f', 'when the oil is hot , dredge chicken wings first in the egg mixture then the flour', 'fry in small batches for 8-10 minutes , being careful not to crowd , turning occasionally , until cooked through and golden', 'drain on paper toweling or brown paper bags , sprinkling with reserved spice for added spice level if desired', 'when oil reheats to 350f , add a few more wings to the fryer', 'repeat until all wings are cooked', 'serve with spicy dipping sauce']","['chicken wings', 'vegetable oil', 'seasoning salt', 'crushed red pepper flakes', 'black pepper', 'cayenne pepper', 'poultry seasoning', 'lemon pepper', 'eggs', 'hot sauce', 'all-purpose flour', 'sour cream', 'horseradish', 'chili sauce', 'salt', 'fresh ground black pepper']",NaN,4.855004
154214,pat s tomato bread soup,80,"['put olive oil in a soup pot and add crushed garlic and bread', 'saute bread until golden', 'add beef broth and tomatoes', 'cover and simmer for 30 minutes', 'remove cover and simmer an additional 40 minutes', 'just before serving , add fresh basil leaves or spinach leaves', 'cooking time is approximate']","['olive oil', 'garlic cloves', 'bread', 'beef broth', 'tomatoes', 'fresh basil leaves']",NaN,4.855004
154215,pat in pie crust no roll,15,"['sift together dry ingredients', 'stir together wet ingredients', 'combine , stirring just to blend', 'press and pat into a 9 inch pan , up the sides , crimping the edges', 'bake as directed in recipe']","['flour', 'sugar', 'baking powder', 'salt', 'vanilla', 'light olive oil', 'cold water']",NaN,4.855004
154216,pat in pan crust,40,"['place the flour , sugar , & salt in the pie pan and mix with your fingers until blended', 'in a measuring cup , combine the oil & milk and beat with a fork until creamy', 'pour all at once over the flour mixture', 'mix with a fork until the flour mixture is completely moistened', 'pat the dough with your fingers , first up the side of the 